In [3]:
import cv2  
import base64
from openai import OpenAI

client = OpenAI(api_key='***REMOVED***')

def query_model(video_path, prompt):
    video = cv2.VideoCapture(video_path)
    
    base64Frames = []
    while video.isOpened():
        success, frame = video.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

    PROMPT_MESSAGES = [
        {
            "role": "user",
            "content": [
                prompt,
                *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::50]),
            ],
            'detail': 'high',
        },
    ]
    params = {
        "model": "gpt-4o-mini",
        "messages": PROMPT_MESSAGES,
    }

    result = client.chat.completions.create(**params)
    return result.choices[0].message.content

In [ ]:
prompt = '''
Was ego doing a legal maneuver if its goal is to turn right at the intersection? 

Options:

(A) It's legal as the lane is empty. (B) It's illegal as the right turn lane is bloacked by construction. (C) It's illegal as ego was cutting in other vehicles that were waiting. (D) It's legal but the lane ahead is way too narrow for ego to pass.
'''

video_path = 'tesla-real-world-video-q-a/videos/videos/00001.mp4'

response = query_model(video_path, prompt)
response

"Based on the context and the provided options, it seems the most fitting answer for whether Ego is doing a legal maneuver to turn right at the intersection is:\n\n**(B)** It's illegal as the right turn lane is blocked by construction.\n\nThe images clearly show construction barriers that obstruct the right turn lane, indicating it's likely not legal to attempt that maneuver."